## Influence of hyper-paramters 

In [61]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
import matplotlib.pyplot as plt
import numpy as np
from collections import defaultdict
from tqdm import tqdm
import os
from sklearn.metrics import precision_score, recall_score, f1_score, confusion_matrix
from common import VGGStyleCNN, load_data
from pprint import pprint
import warnings
warnings.filterwarnings('ignore')

In [45]:
def train_model(train_data, directory, num_epochs=10, learning_rate=0.001, dropout_rate=0.5, weight_decay=0, batch_size=128):
    train_accuracy = []
    train_loss = []
    
    
    device = torch.device("xpu" if torch.xpu.is_available() else "cpu")
    model = VGGStyleCNN(dropout_rate=dropout_rate).to(device)

    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=learning_rate, weight_decay=weight_decay)

    for epoch in range(num_epochs):
        model.train()
        running_loss = 0.0
        correct = 0
        total = 0

        train_loader_tqdm = tqdm(train_data, desc=f"Epoch {epoch+1}/{num_epochs}")

        for images, labels in train_loader_tqdm:
            images, labels = images.to(device), labels.to(device)

            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            running_loss += loss.item()

            # Calculate accuracy
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

            train_loader_tqdm.set_postfix(loss=loss.item())

        avg_loss = running_loss / len(train_data)
        accuracy = 100 * correct / total

        train_accuracy.append(accuracy)
        train_loss.append(avg_loss)
        
        print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {avg_loss:.4f}, Accuracy: {accuracy:.2f}%")

    # Save the model
    model_save_path = f'models/{directory}/lr_{learning_rate}_dropout_{dropout_rate}_wd_{weight_decay}_bs_{batch_size}'
    os.makedirs(os.path.dirname(model_save_path), exist_ok=True)  # Create directory if it doesn't exist
    torch.save(model.state_dict(), model_save_path)
    print(f"Model saved at: {model_save_path} \n")
    
    return {
        'model': model,
        'accuracy': train_accuracy,
        'loss': train_loss,
    }

In [31]:
def test_model(test_data , model=None, model_path=''):
    device = torch.device("xpu" if torch.xpu.is_available() else "cpu")
    
    if model_path:
        torch.load(model_path, map_location=torch.device('xpu'))
        model.load_state_dict(torch.load(model_path))
    
    model.eval()
    
    correct = 0
    total = 0
    all_predictions = []
    all_labels = []

    with torch.no_grad():
        for images, labels in test_data:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            _, predicted = torch.max(outputs, 1)

            total += labels.size(0)
            correct += (predicted == labels).sum().item()

            all_predictions.extend(predicted.cpu().numpy())  # Collect predictions
            all_labels.extend(labels.cpu().numpy())  # Collect true labels

    # Calculate accuracy
    accuracy = 100 * correct / total
    print(f"Accuracy on validation set: {accuracy:.2f}%")

    # Calculate additional metrics
    precision = precision_score(all_labels, all_predictions, average='weighted')
    recall = recall_score(all_labels, all_predictions, average='weighted')
    f1 = f1_score(all_labels, all_predictions, average='weighted')
    conf_matrix = confusion_matrix(all_labels, all_predictions)

    return {
        'accuracy': accuracy,
        'precision': precision,
        'recall': recall,
        'f1': f1,
        'conf_matrix': conf_matrix
    }

### Training process

#### Learning rate

In [37]:
# loading the data
loaded_data = load_data()  # default batch_size = 128
train_data, test_data = loaded_data['train'], loaded_data['test']

In [38]:
# Training models
learning_rates = [0.00001, 0.0001, 0.001, 0.01, 0.1]
models_lr = []


for lr in learning_rates:
    print(f'------Training model with learning_rate: {lr}-------')
    train_results = train_model(train_data, directory='lr', num_epochs=10, learning_rate=lr)
    models_lr.append(train_results)

------Training model with learning_rate: 1e-05-------


Epoch 1/10: 100%|██████████| 704/704 [02:48<00:00,  4.18it/s, loss=1.69]


Epoch [1/10], Loss: 1.7452, Accuracy: 34.46%


Epoch 2/10: 100%|██████████| 704/704 [02:48<00:00,  4.18it/s, loss=1.37]


Epoch [2/10], Loss: 1.4202, Accuracy: 47.46%


...

Epoch [8/10], Loss: 2.3088, Accuracy: 10.01%


Epoch 9/10: 100%|██████████| 704/704 [02:23<00:00,  4.92it/s, loss=2.3] 


Epoch [9/10], Loss: 2.3163, Accuracy: 9.99%


Epoch 10/10: 100%|██████████| 704/704 [02:23<00:00,  4.92it/s, loss=2.3] 


Epoch [10/10], Loss: 2.3088, Accuracy: 10.06%
Model saved at: models/lr/lr_0.1_dropout_0.5_wd_0_bs_128 



In [40]:
tests_lr = []
for train_results, lr in zip(models_lr, learning_rates):
    print(f"-----Results for learning rate: {lr}-----")
    test_results = test_model(test_data, train_results['model'])
    tests_lr.append(test_results)
    pprint(test_results)
    print()

-----Results for learning rate: 1e-05-----
Accuracy on validation set: 53.44%
{'accuracy': 53.43555555555555,
 'conf_matrix': array([[5990,  441,  582,  161,  193,  164,   96,  162,  915,  296],
       [ 310, 6042,   92,  143,  129,  188,  106,  201,  398, 1391],
       [ 559,  140, 4131,  940,  972,  719,  857,  270,  340,   72],
       [ 121,  157,  763, 3604, 1233, 1524,  909,  404,  204,   81],
       [ 210,  118,  936,  947, 4127,  804,  439, 1126,  216,   77],
       [ 122,  187,  785, 1880, 1423, 3016,  390,  879,  219,   99],
       [  50,   98,  750, 1058,  562,  355, 5907,   70,  118,   32],
       [ 171,  155,  341,  416, 1295,  844,   85, 5372,  145,  176],
       [1023,  712,  376,  304,  273,  241,  171,  178, 5222,  500],
       [ 394, 2229,  102,  233,  187,  251,   89,  341,  493, 4681]]),
 'f1': 0.5347587663123766,
 'precision': 0.5377690378257005,
 'recall': 0.5343555555555556}

-----Results for learning rate: 0.0001-----
Accuracy on validation set: 68.37%
{'accuracy

c:\Users\zp13279\.conda\envs\pytorch-intel\Lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


Accuracy on validation set: 10.01%
{'accuracy': 10.006666666666666,
 'conf_matrix': array([[   4,    0,    0,    0,    0,    0,    0, 8961,   35,    0],
       [   1,    0,    0,    0,    0,    0,    0, 8997,    2,    0],
       [   1,    0,    0,    0,    0,    0,    0, 8972,   27,    0],
       [   0,    0,    0,    0,    0,    0,    0, 8992,    8,    0],
       [   0,    0,    0,    0,    0,    0,    0, 8999,    1,    0],
       [   0,    0,    0,    0,    0,    0,    0, 8986,   14,    0],
       [   0,    0,    0,    0,    0,    0,    0, 8978,   22,    0],
       [   0,    0,    0,    0,    0,    0,    0, 8998,    2,    0],
       [   0,    0,    0,    0,    0,    0,    0, 8996,    4,    0],
       [   1,    0,    0,    0,    0,    0,    0, 8997,    2,    0]]),
 'f1': 0.01837714242648527,
 'precision': 0.0705732320601696,
 'recall': 0.10006666666666666}



c:\Users\zp13279\.conda\envs\pytorch-intel\Lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


#### Batch size

In [43]:
batch_sizes = [16, 32, 64, 128, 256]
models_bs = []
test_data_bs = []

In [46]:
for bs in batch_sizes:
    loaded_data = load_data(batch_size=bs)
    train_data, test_data = loaded_data['train'], loaded_data['test']
    test_data_bs.append(test_data)

    print(f'------Training model with batch size: {bs}-------')
    train_results = train_model(train_data, directory='bs', num_epochs=10, batch_size=bs)
    models_bs.append(train_results)
    

------Training model with batch size: 16-------


Epoch 1/10: 100%|██████████| 5625/5625 [05:57<00:00, 15.74it/s, loss=1.84]


Epoch [1/10], Loss: 1.9999, Accuracy: 25.70%


Epoch 2/10: 100%|██████████| 5625/5625 [05:55<00:00, 15.84it/s, loss=1.55] 


Epoch [2/10], Loss: 1.6911, Accuracy: 36.74%


...

Epoch [8/10], Loss: 0.5525, Accuracy: 80.73%


Epoch 9/10: 100%|██████████| 352/352 [02:12<00:00,  2.66it/s, loss=0.437]


Epoch [9/10], Loss: 0.5195, Accuracy: 82.30%


Epoch 10/10: 100%|██████████| 352/352 [02:12<00:00,  2.66it/s, loss=1.66] 

Epoch [10/10], Loss: 0.8709, Accuracy: 73.41%
Model saved at: models/bs/lr_0.001_dropout_0.5_wd_0_bs_256 



In [47]:
tests_bs = []
for train_results, bs, test_data in zip(models_bs, batch_sizes, test_data_bs):
    print(f"-----Results for batch size: {bs}-----")
    test_results = test_model(test_data, train_results['model'])
    tests_bs.append(test_results)
    pprint(test_results)
    print()

-----Results for batch size: 16-----
Accuracy on validation set: 64.05%
{'accuracy': 64.04888888888888,
 'conf_matrix': array([[7188,   83,  822,   46,   19,   26,   21,   95,  539,  161],
       [ 674, 6147,   77,   58,   13,   29,   62,   81,  294, 1565],
       [ 717,   21, 6117,  477,  353,  226,  721,  130,  207,   31],
       [ 158,   35,  986, 4959,  536,  983,  870,  187,  175,  111],
       [ 467,   12, 1353,  706, 4376,  363,  396, 1105,  164,   58],
       [ 290,   39, 1100, 2074,  947, 3172,  326,  805,  130,  117],
       [ 412,   26,  987,  666,   82,   61, 6641,   12,   97,   16],
       [ 541,   22,  418,  268,  603,  490,   45, 6356,  100,  157],
       [1245,  201,  528,  141,   30,   39,  107,   81, 6167,  461],
       [ 516, 1189,  108,   73,   35,   28,   44,  141,  345, 6521]]),
 'f1': 0.6372113045388966,
 'precision': 0.6489353329443136,
 'recall': 0.6404888888888889}

-----Results for batch size: 32-----
Accuracy on validation set: 69.39%
{'accuracy': 69.3866666

### Regularization process

#### Drop out rate

In [48]:
loaded_data = load_data()
train_data, test_data = loaded_data['train'], loaded_data['test']

In [49]:
drop_out_rates = [0.1, 0.2, 0.3, 0.5]
models_dr = []

for dr in drop_out_rates:
    print(f'------Training model with drop out rate: {dr}-------')
    train_results = train_model(train_data, directory='dr', num_epochs=10, dropout_rate=dr)
    models_dr.append(train_results)

------Training model with drop out rate: 0.1-------


Epoch 1/10: 100%|██████████| 704/704 [02:26<00:00,  4.81it/s, loss=1.15]


Epoch [1/10], Loss: 1.5661, Accuracy: 41.87%


Epoch 2/10: 100%|██████████| 704/704 [02:25<00:00,  4.85it/s, loss=1.08] 


Epoch [2/10], Loss: 1.1774, Accuracy: 57.53%


...

Epoch [8/10], Loss: 0.9062, Accuracy: 68.50%


Epoch 9/10: 100%|██████████| 704/704 [02:23<00:00,  4.92it/s, loss=1.29] 


Epoch [9/10], Loss: 0.7978, Accuracy: 72.42%


Epoch 10/10: 100%|██████████| 704/704 [02:23<00:00,  4.91it/s, loss=0.702]


Epoch [10/10], Loss: 0.7057, Accuracy: 75.30%
Model saved at: models/dr/lr_0.001_dropout_0.5_wd_0_bs_128 



In [50]:
tests_dr = []
for train_results, dr in zip(models_dr, drop_out_rates):
    print(f"-----Results for drop out rate: {dr}-----")
    test_results = test_model(test_data, train_results['model'])
    tests_dr.append(test_results)
    pprint(test_results)
    print()

-----Results for drop out rate: 0.1-----
Accuracy on validation set: 73.39%
{'accuracy': 73.38666666666667,
 'conf_matrix': array([[7551,   86,  212,   21,   55,   30,   18,  134,  732,  161],
       [ 169, 6852,   21,   38,   15,   67,   16,   86,  254, 1482],
       [ 431,   38, 6010,  432,  469,  625,  331,  253,  359,   52],
       [ 163,   47,  318, 5145,  464, 2003,  227,  379,  148,  106],
       [ 177,   29,  346,  464, 5487, 1020,   85, 1200,  109,   83],
       [ 135,   59,  299,  882,  608, 5569,  101, 1083,  147,  117],
       [  49,   61,  535,  589,  170,  310, 7049,   59,  125,   53],
       [ 129,   49,  102,  144,  291,  374,   12, 7681,   93,  125],
       [ 411,  181,  145,   75,   55,   80,   36,  120, 7578,  319],
       [ 259, 1028,   34,   46,   37,   61,   15,  141,  253, 7126]]),
 'f1': 0.7331164142268867,
 'precision': 0.7379053973039135,
 'recall': 0.7338666666666667}

-----Results for drop out rate: 0.2-----
Accuracy on validation set: 73.88%
{'accuracy': 73

#### Weight decay (L2 regularization)

In [51]:
weight_decays = [0.0, 0.0001, 0.001, 0.01]
models_wd = []

for wd in weight_decays:
    print(f'------Training model with weight decay: {wd}-------')
    train_results = train_model(train_data, directory='wd', num_epochs=10, weight_decay=wd, dropout_rate=0.3)
    models_wd.append(train_results)

------Training model with weight decay: 0.0-------


Epoch 1/10: 100%|██████████| 704/704 [02:26<00:00,  4.81it/s, loss=1.69]


Epoch [1/10], Loss: 1.5818, Accuracy: 41.43%


Epoch 2/10: 100%|██████████| 704/704 [02:24<00:00,  4.86it/s, loss=1.32] 


Epoch [2/10], Loss: 1.2180, Accuracy: 56.15%


...

Epoch [8/10], Loss: 1.1463, Accuracy: 58.82%


Epoch 9/10: 100%|██████████| 704/704 [02:26<00:00,  4.82it/s, loss=1.11] 


Epoch [9/10], Loss: 1.1233, Accuracy: 59.63%


Epoch 10/10: 100%|██████████| 704/704 [02:26<00:00,  4.81it/s, loss=1.03] 


Epoch [10/10], Loss: 1.1088, Accuracy: 60.40%
Model saved at: models/wd/lr_0.001_dropout_0.3_wd_0.01_bs_128 



In [52]:
tests_wd = []
for train_results, wd in zip(models_wd, weight_decays):
    print(f"-----Results for weight decay: {wd}-----")
    test_results = test_model(test_data, train_results['model'])
    tests_wd.append(test_results)
    pprint(test_results)
    print()

-----Results for weight decay: 0.0-----
Accuracy on validation set: 72.42%
{'accuracy': 72.41666666666667,
 'conf_matrix': array([[6971,  104,  420,   93,   97,   24,   15,  173,  745,  358],
       [  80, 6244,   29,   71,   53,   64,   10,   84,  108, 2257],
       [ 237,   22, 6393,  640,  600,  339,  383,  149,  178,   59],
       [  60,   37,  393, 6102,  907,  901,  199,  209,   79,  113],
       [  61,   22,  486,  519, 6541,  377,  137,  689,   84,   84],
       [  59,   50,  378, 1802, 1265, 4387,   58,  781,   96,  124],
       [  25,   61,  524, 1013,  272,  179, 6807,   32,   54,   33],
       [  66,   26,  142,  221,  762,  305,    8, 7225,   55,  190],
       [ 268,  260,  228,  136,  171,   46,   42,  146, 6921,  782],
       [ 142,  782,   32,   79,   87,   29,   18,  145,  102, 7584]]),
 'f1': 0.7249189467156616,
 'precision': 0.7356362952348933,
 'recall': 0.7241666666666666}

-----Results for weight decay: 0.0001-----
Accuracy on validation set: 72.39%
{'accuracy': 7

In [55]:
# saving the models and training results
base_file_path = 'C:/Users/zp13279/Desktop/Repos_other/DeepLearning/kod/training_results'
os.makedirs(base_file_path, exist_ok=True)

model_results = [models_lr, models_bs, models_dr, models_wd]
param_names = ['models_lr.txt', 'models_bs.txt', 'models_dr.txt', 'models_wd.txt']

# Saving the models and training results
for data, param_name in zip(model_results, param_names):
    file_path = f"{base_file_path}/{param_name}"  # Create the full file path
    with open(file_path, 'w') as file:
        for entry in data:
            file.write(str(entry) + '\n')

In [56]:
# saving the validation results
base_file_path = 'C:/Users/zp13279/Desktop/Repos_other/DeepLearning/kod/validation_results'
os.makedirs(base_file_path, exist_ok=True)

model_validation_results = [tests_lr, tests_bs, tests_dr, tests_wd]
param_names = ['tests_lr.txt', 'tests_bs.txt', 'tests_dr.txt', 'tests_wd.txt']

# Saving the models and training results
for data, param_name in zip(model_validation_results, param_names):
    file_path = f"{base_file_path}/{param_name}"  # Create the full file path
    with open(file_path, 'w') as file:
        for entry in data:
            file.write(str(entry) + '\n')

## Few-shot learning approach

In [62]:
def load_data_few_shot(batch_size=128, few_shot_samples_per_class=None):
    cinic_directory = 'C:/Users/zp13279/Desktop/Repos_other/DeepLearning/kod/data'
    cinic_mean = [0.47889522, 0.47227842, 0.43047404]
    cinic_std = [0.24205776, 0.23828046, 0.25874835]

    torch.manual_seed(42)

    train_dataset = torchvision.datasets.ImageFolder(
        cinic_directory + '/train',
        transform=transforms.Compose([
            transforms.ToTensor(),
            transforms.Normalize(mean=cinic_mean, std=cinic_std)
        ])
    )
    
    if few_shot_samples_per_class is not None:
        # Create a few-shot dataset
        few_shot_data = []
        class_count = defaultdict(int)
        
        for img, label in train_dataset:
            if class_count[label] < few_shot_samples_per_class:
                few_shot_data.append((img, label))
                class_count[label] += 1
                
            # Stop if we have enough samples for all classes
            if all(count >= few_shot_samples_per_class for count in class_count.values()):
                break

        train_dataset = few_shot_data

    cinic_train = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

    cinic_test = DataLoader(
        torchvision.datasets.ImageFolder(cinic_directory + '/valid',
        transform=transforms.Compose([
            transforms.ToTensor(),
            transforms.Normalize(mean=cinic_mean, std=cinic_std)
        ])),
        batch_size=batch_size, shuffle=False
    )
    
    return {
        'train': cinic_train,
        'test': cinic_test
    }

In [65]:
samples_number = [500, 1000, 2000, 3000, 4000]
models_few_shot = []
tests_few_shot = []


In [66]:
for samples_no in samples_number:
    loaded_shot_data = load_data_few_shot(batch_size=64, few_shot_samples_per_class=samples_no)
    train_data, test_data = loaded_shot_data['train'], loaded_shot_data['test']
    tests_few_shot.append(test_data)

    print(f'------Training model with: {samples_no} samples-------')
    train_results = train_model(train_data, directory='few_shot', num_epochs=10, batch_size=64)
    models_few_shot.append(train_results)

------Training model with: 500 samples-------


Epoch 1/10: 100%|██████████| 8/8 [00:03<00:00,  2.62it/s, loss=0]  


Epoch [1/10], Loss: 0.3121, Accuracy: 87.20%


Epoch 2/10: 100%|██████████| 8/8 [00:01<00:00,  7.92it/s, loss=0]


Epoch [2/10], Loss: 0.0000, Accuracy: 100.00%


...

Epoch [8/10], Loss: 0.0000, Accuracy: 100.00%


Epoch 9/10: 100%|██████████| 63/63 [00:07<00:00,  8.05it/s, loss=0]


Epoch [9/10], Loss: 0.0000, Accuracy: 100.00%


Epoch 10/10: 100%|██████████| 63/63 [00:07<00:00,  8.07it/s, loss=0]


Epoch [10/10], Loss: 0.0000, Accuracy: 100.00%
Model saved at: models/few_shot/lr_0.001_dropout_0.5_wd_0_bs_64 



In [68]:
tests_few_shot_results = []
for train_results, sn, test_data in zip(models_few_shot, samples_number, tests_few_shot):
    print(f"-----Results for samples number {sn}-----")
    test_results = test_model(test_data, train_results['model'])
    tests_few_shot_results.append(test_results)
    pprint(test_results)
    print()

-----Results for samples number 500-----
Accuracy on validation set: 10.00%
{'accuracy': 10.0,
 'conf_matrix': array([[9000,    0,    0,    0,    0,    0,    0,    0,    0,    0],
       [9000,    0,    0,    0,    0,    0,    0,    0,    0,    0],
       [9000,    0,    0,    0,    0,    0,    0,    0,    0,    0],
       [9000,    0,    0,    0,    0,    0,    0,    0,    0,    0],
       [9000,    0,    0,    0,    0,    0,    0,    0,    0,    0],
       [9000,    0,    0,    0,    0,    0,    0,    0,    0,    0],
       [9000,    0,    0,    0,    0,    0,    0,    0,    0,    0],
       [9000,    0,    0,    0,    0,    0,    0,    0,    0,    0],
       [9000,    0,    0,    0,    0,    0,    0,    0,    0,    0],
       [9000,    0,    0,    0,    0,    0,    0,    0,    0,    0]]),
 'f1': 0.018181818181818184,
 'precision': 0.01,
 'recall': 0.1}

-----Results for samples number 1000-----
Accuracy on validation set: 10.00%
{'accuracy': 10.0,
 'conf_matrix': array([[9000,    0,